# Experiment Runner — Automated Batch
Runs zero-filled and real/complex U-Nets sequentially so overnight jobs log everything needed for the paper.


## Workflow
1. Configure experiments in the cell below.
2. Execute the final cell to iterate through each run.
3. Inspect `results/` in the morning for metrics, checkpoints, and figures.


In [5]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
from pathlib import Path
import csv
import json
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import torch
import yaml
from torch.utils.data import DataLoader, Subset
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_ROOT = PROJECT_ROOT / 'src'
RESULTS_ROOT = PROJECT_ROOT / 'results'
RESULTS_ROOT.mkdir(exist_ok=True)
import sys
if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))
from data.dataset import SingleCoilDataset
from data.masking import EquispacedMasker
from models.real_unet import RealUnet
from models.cx_unet import ComplexUnet
from training.utils import train_loop, test_loop


In [6]:
LOG_EVERY = 1


def init_run(run_tag: str, config: dict) -> Path:
    timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
    # run_dir = RESULTS_ROOT / f"{timestamp}_{run_tag}"
    run_dir = RESULTS_ROOT / run_tag
    (run_dir / 'checkpoints').mkdir(parents=True, exist_ok=True)
    (run_dir / 'qualitative').mkdir(exist_ok=True)
    (run_dir / 'metrics').mkdir(exist_ok=True)
    (run_dir / 'images').mkdir(exist_ok=True)
    (run_dir / 'tensors').mkdir(exist_ok=True)
    with open(run_dir / 'config.yaml', 'w') as f:
        yaml.safe_dump(config, f)
    with open(run_dir / 'config.json', 'w') as f:
        json.dump(config, f, indent=2)
    return run_dir


def to_mag(x: torch.Tensor) -> torch.Tensor:
    return x.abs()


def psnr_db(x: torch.Tensor, y: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    mse = torch.mean((x - y) ** 2)
    return 10.0 * torch.log10(1.0 / (mse + eps))


def ssim_simple(x: torch.Tensor, y: torch.Tensor, C1: float = 0.01**2, C2: float = 0.03**2) -> torch.Tensor:
    mu_x, mu_y = x.mean(), y.mean()
    sigma_x = ((x - mu_x) ** 2).mean()
    sigma_y = ((y - mu_y) ** 2).mean()
    sigma_xy = ((x - mu_x) * (y - mu_y)).mean()
    num = (2 * mu_x * mu_y + C1) * (2 * sigma_xy + C2)
    den = (mu_x**2 + mu_y**2 + C1) * (sigma_x + sigma_y + C2)
    return num / (den + 1e-8)


@torch.no_grad()
def evaluate_metrics(model, dataloader, device, num_batches: int | None = None):
    model.eval()
    psnrs, ssim_scores, l1s = [], [], []
    for batch_idx, (masked, target) in enumerate(dataloader):
        masked = masked.to(device)
        target = target.to(device)
        recon = model(masked)
        gt = to_mag(target)
        pred = to_mag(recon)
        gt = gt / gt.max().clamp_min(1e-8)
        pred = pred / pred.max().clamp_min(1e-8)
        psnrs.append(psnr_db(pred, gt).item())
        ssim_scores.append(ssim_simple(pred, gt).item())
        l1s.append(torch.mean(torch.abs(pred - gt)).item())
        if num_batches is not None and (batch_idx + 1) >= num_batches:
            break
    return {
        'psnr': float(sum(psnrs) / max(len(psnrs), 1)),
        'ssim': float(sum(ssim_scores) / max(len(ssim_scores), 1)),
        'l1': float(sum(l1s) / max(len(l1s), 1)),
    }


@torch.no_grad()
def save_qualitative(run_dir: Path, model, dataset, device, epoch: int, sample_idx: int | None = None):
    model.eval()
    if len(dataset) == 0:
        return
    idx = sample_idx if sample_idx is not None else (epoch * 11) % len(dataset)
    masked, target = dataset[idx]
    masked = masked.unsqueeze(0).to(device)
    target = target.unsqueeze(0).to(device)
    recon = model(masked)
    gt = to_mag(target[0]).cpu()
    pred = to_mag(recon[0]).detach().cpu()
    zf = to_mag(masked[0]).cpu()
    for img in (gt, pred, zf):
        img /= img.max().clamp_min(1e-8)
    plt.imsave(run_dir / 'qualitative' / f'epoch{epoch:03d}_zf.png', zf.squeeze(), cmap='gray')
    plt.imsave(run_dir / 'qualitative' / f'epoch{epoch:03d}_pred.png', pred.squeeze(), cmap='gray')
    plt.imsave(run_dir / 'qualitative' / f'epoch{epoch:03d}_gt.png', gt.squeeze(), cmap='gray')
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    for ax, img, title in zip(axes, [zf, pred, gt], ['Zero-filled', 'Model', 'Ground Truth']):
        ax.imshow(img.squeeze(), cmap='gray')
        ax.set_title(title)
        ax.axis('off')
    fig.suptitle(f'Epoch {epoch}')
    fig.tight_layout()
    fig.savefig(run_dir / 'qualitative' / f'epoch{epoch:03d}.png', dpi=200)
    plt.close(fig)


def select_indices(n_total: int, n_pick: int) -> np.ndarray:
    n = min(n_total, n_pick)
    if n_total == 0:
        return np.array([], dtype=int)
    return np.linspace(0, n_total - 1, num=n, dtype=int)


def save_zero_fill_outputs(cfg: dict, run_dir: Path):
    accels = cfg['mask_grid']['accels']
    acs_list = cfg['mask_grid']['acs']
    idx = cfg.get('grid_index', 0)
    rows = []
    for accel in accels:
        for acs in acs_list:
            ds = SingleCoilDataset(cfg['val_folder'], mask_func=EquispacedMasker(accel=accel, acs=acs))
            slice_idx = min(idx, len(ds) - 1)
            masked, target = ds[slice_idx]
            gt = to_mag(target).float()
            zf = to_mag(masked).float()
            gt = gt / gt.max().clamp_min(1e-8)
            zf = zf / zf.max().clamp_min(1e-8)
            psnr = psnr_db(zf, gt).item()
            ssim = ssim_simple(zf, gt).item()
            l1 = torch.mean(torch.abs(zf - gt)).item()
            rows.append({'accel': accel, 'acs': acs, 'idx': int(slice_idx), 'psnr': psnr, 'ssim': ssim, 'l1': l1})
            base = run_dir / 'images' / f'idx{slice_idx:04d}_R{accel}_ACS{acs}'
            plt.imsave(str(base) + '_zf.png', zf.squeeze(), cmap='gray')
            plt.imsave(str(base) + '_gt.png', gt.squeeze(), cmap='gray')
    df = pd.DataFrame(rows)
    df.to_csv(run_dir / 'zero_fill_metrics.csv', index=False)
    (run_dir / 'zero_fill_metrics.json').write_text(json.dumps(rows, indent=2))
    display(df)


def build_subset(dataset, subset_size):
    if subset_size is None:
        return dataset
    n = min(len(dataset), subset_size)
    indices = list(range(n))
    return Subset(dataset, indices)


def train_epoch_with_logs(model, dataloader, optimizer, loss_fn, device, epoch: int, global_step: int, log_every: int = 100):
    model.train()
    total_loss = 0.0
    step_logs = []
    progress = tqdm(dataloader, desc=f"train epoch {epoch}", leave=False)
    for step, (images_masked, images_full) in enumerate(progress, start=1):
        images_masked = images_masked.to(device)
        images_full = images_full.to(device)
        optimizer.zero_grad()
        outputs = model(images_masked)
        loss = loss_fn(outputs, images_full)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        global_step += 1
        if global_step % log_every == 0:
            step_logs.append({'global_step': global_step, 'epoch': epoch, 'step': step, 'loss': loss.item()})
    avg_loss = total_loss / max(len(dataloader), 1)
    return avg_loss, global_step, step_logs


@torch.no_grad()
def validate_epoch(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    progress = tqdm(dataloader, desc="val", leave=False)
    for images_masked, images_full in progress:
        images_masked = images_masked.to(device)
        images_full = images_full.to(device)
        outputs = model(images_masked)
        loss = loss_fn(outputs, images_full)
        total_loss += loss.item()
    return total_loss / max(len(dataloader), 1)


def run_zero_fill_experiment(cfg: dict):
    run_dir = init_run(cfg['run_tag'], cfg)
    save_zero_fill_outputs(cfg, run_dir)


def run_model_experiment(cfg: dict):
    torch.manual_seed(cfg.get('seed', 0))
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
    run_dir = init_run(cfg['run_tag'], cfg)
    mask_cfg = cfg.get('mask', {'accel': 4, 'acs': 24})
    masker = EquispacedMasker(accel=mask_cfg['accel'], acs=mask_cfg['acs'])
    train_full = SingleCoilDataset(cfg['train_folder'], mask_func=masker)
    val_full = SingleCoilDataset(cfg['val_folder'], mask_func=masker)
    train_set = build_subset(train_full, cfg.get('train_subset'))
    val_set = build_subset(val_full, cfg.get('val_subset'))
    train_loader = DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True, num_workers=cfg['num_workers'])
    val_loader = DataLoader(val_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=cfg['num_workers'])
    features = cfg.get('features', [32, 64, 128, 256, 512])
    if cfg.get('model', 'real') == 'complex':
        model = ComplexUnet(in_channels=1, out_channels=1, features=features).to(device)
    else:
        model = RealUnet(in_channels=1, out_channels=1, features=features, width_scale=cfg.get('width_scale', 1.0)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    loss_fn = lambda pred, target: (pred - target).abs().mean()

    step_path = run_dir / 'metrics' / 'step_metrics.csv'
    epoch_path = run_dir / 'metrics' / 'epoch_metrics.csv'
    with step_path.open('w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['global_step', 'epoch', 'step', 'loss'])
        writer.writeheader()
    with epoch_path.open('w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['epoch', 'train_loss', 'val_loss', 'psnr', 'ssim', 'l1'])
        writer.writeheader()
    summary_path = run_dir / 'metrics' / 'summary.json'

    best_val = float('inf')
    best_epoch = 0
    global_step = 0
    qual_idx = cfg.get('qualitative_index', 0)

    for epoch in range(1, cfg['epochs'] + 1):
        train_loss, global_step, step_logs = train_epoch_with_logs(model, train_loader, optimizer, loss_fn, device, epoch, global_step, log_every=LOG_EVERY)
        if step_logs:
            with step_path.open('a', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=['global_step', 'epoch', 'step', 'loss'])
                writer.writerows(step_logs)
        val_loss = validate_epoch(model, val_loader, loss_fn, device)
        metrics = evaluate_metrics(model, val_loader, device, num_batches=None)
        with epoch_path.open('a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=['epoch', 'train_loss', 'val_loss', 'psnr', 'ssim', 'l1'])
            writer.writerow({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss, **metrics})
        summary = {
            'best_val_loss': float(min(best_val, val_loss)),
            'best_epoch': int(best_epoch if val_loss >= best_val else epoch),
            'last_epoch': int(epoch),
            'device': str(device),
            'global_step': int(global_step),
        }
        summary_path.write_text(json.dumps(summary, indent=2))
        torch.save({'model': model.state_dict(), 'epoch': epoch}, run_dir / 'checkpoints' / 'latest.pt')
        if val_loss < best_val:
            best_val = val_loss
            best_epoch = epoch
            torch.save({'model': model.state_dict(), 'epoch': epoch}, run_dir / 'checkpoints' / 'best.pt')
        save_qualitative(run_dir, model, val_set, device, epoch, sample_idx=qual_idx)
        print(f"[{cfg['run_tag']}] Epoch {epoch}/{cfg['epochs']} | train {train_loss:.4f} | val {val_loss:.4f} | psnr {metrics['psnr']:.2f} | ssim {metrics['ssim']:.3f} | l1 {metrics['l1']:.4f}")


def run_experiment(cfg: dict):
    exp_type = cfg['type']
    print(f"=== Running {cfg['run_tag']} ({exp_type}) ===")
    if exp_type == 'zero_fill':
        run_zero_fill_experiment(cfg)
    elif exp_type == 'model':
        run_model_experiment(cfg)
    else:
        raise ValueError(f'Unknown experiment type: {exp_type}')


In [7]:
EXPERIMENTS = [
    {
        'run_tag': 'zerofill_sweep',
        'type': 'zero_fill',
        'val_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_val'),
        'val_subset': 64,
        'grid_index': 50,
        'mask_grid': {'accels': [2, 4, 6, 8], 'acs': [16, 20, 24]},
    },

    {
        'run_tag': 'realunet_R6_seed0',
        'type': 'model',
        'model': 'real',
        'train_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_train'),
        'val_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_val'),
        'mask': {'accel': 6, 'acs': 20},
        'train_subset': 16384,
        'val_subset': 1024,
        'batch_size': 4, 
        'num_workers': 4,
        'epochs': 12,
        'learning_rate': 1e-3,
        'width_scale': 1.42,
        'features': [16, 32, 64, 128, 256],
        'seed': 0,
        'qualitative_index': 50,
    },
    {
        'run_tag': 'complexunet_R6_seed0',
        'type': 'model',
        'model': 'complex',
        'train_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_train'),
        'val_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_val'),
        'mask': {'accel': 6, 'acs': 20},
        'train_subset': 16384,
        'val_subset': 1024,
        'batch_size': 4,
        'num_workers': 4,
        'epochs': 12,
        'learning_rate': 1e-3,
        'features': [16, 32, 64, 128, 256],
        'seed': 0,
        'qualitative_index': 50,
    },
]

# Quick sanity runs (much smaller) to validate logging/IO without long training.
QUICK_EXPERIMENTS = [
    {
        'run_tag': 'zerofill_quick',
        'type': 'zero_fill',
        'val_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_val'),
        'val_subset': 4,
        'grid_index': 0,
        'mask_grid': {'accels': [6], 'acs': [20]},
    },
    {
        'run_tag': 'realunet_quick',
        'type': 'model',
        'model': 'real',
        'train_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_train'),
        'val_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_val'),
        'mask': {'accel': 6, 'acs': 20},
        'train_subset': 64,
        'val_subset': 16,
        'batch_size': 4,
        'num_workers': 0,
        'epochs': 1,
        'learning_rate': 1e-3,
        'width_scale': 1.42,
        'features': [16, 32, 64, 128, 256],
        'seed': 1,
        'qualitative_index': 0,
    },
    {
        'run_tag': 'complexunet_quick',
        'type': 'model',
        'model': 'complex',
        'train_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_train'),
        'val_folder': str(PROJECT_ROOT / 'data' / 'singlecoil_val'),
        'mask': {'accel': 6, 'acs': 20},
        'train_subset': 64,
        'val_subset': 16,
        'batch_size': 4,
        'num_workers': 0,
        'epochs': 1,
        'learning_rate': 1e-3,
        'features': [16, 32, 64, 128, 256],
        'seed': 1,
        'qualitative_index': 0,
    },
]

USE_QUICK = True  # set True for the fast sanity suite
ACTIVE_EXPERIMENTS = QUICK_EXPERIMENTS if USE_QUICK else EXPERIMENTS
ACTIVE_EXPERIMENTS


[{'run_tag': 'zerofill_quick',
  'type': 'zero_fill',
  'val_folder': '/Users/giodegeronimo/Desktop/ECE570/ece570-tinyreproductions/data/singlecoil_val',
  'val_subset': 4,
  'grid_index': 0,
  'mask_grid': {'accels': [6], 'acs': [20]}},
 {'run_tag': 'realunet_quick',
  'type': 'model',
  'model': 'real',
  'train_folder': '/Users/giodegeronimo/Desktop/ECE570/ece570-tinyreproductions/data/singlecoil_train',
  'val_folder': '/Users/giodegeronimo/Desktop/ECE570/ece570-tinyreproductions/data/singlecoil_val',
  'mask': {'accel': 6, 'acs': 20},
  'train_subset': 64,
  'val_subset': 16,
  'batch_size': 4,
  'num_workers': 0,
  'epochs': 1,
  'learning_rate': 0.001,
  'width_scale': 1.42,
  'features': [16, 32, 64, 128, 256],
  'seed': 1,
  'qualitative_index': 0},
 {'run_tag': 'complexunet_quick',
  'type': 'model',
  'model': 'complex',
  'train_folder': '/Users/giodegeronimo/Desktop/ECE570/ece570-tinyreproductions/data/singlecoil_train',
  'val_folder': '/Users/giodegeronimo/Desktop/ECE570

In [8]:
for cfg in ACTIVE_EXPERIMENTS:
    run_experiment(cfg)


=== Running zerofill_quick (zero_fill) ===


,accel,acs,idx,psnr,ssim,l1
0,6,20,0,16.455532,0.352846,0.115951


=== Running realunet_quick (model) ===


train epoch 1:   0%|          | 0/16 [00:00<?, ?it/s]

val:   0%|          | 0/4 [00:00<?, ?it/s]

[realunet_quick] Epoch 1/1 | train 0.1579 | val 0.2060 | psnr 5.63 | ssim 0.005 | l1 0.4815
=== Running complexunet_quick (model) ===


train epoch 1:   0%|          | 0/16 [00:00<?, ?it/s]

val:   0%|          | 0/4 [00:00<?, ?it/s]

[complexunet_quick] Epoch 1/1 | train 0.3082 | val 0.1123 | psnr 4.52 | ssim 0.042 | l1 0.5895
